In [151]:
import os
import sys
import subprocess
import pandas as pd
import pytube

In [152]:
audioset_labels = pd.read_csv("data/class_labels_indices.csv")
# audioset_unbal_train_df = pd.read_csv("data/audioset/unbalanced_train_segments.csv")
audioset_bal_train = pd.read_csv("data/balanced_train_segments-edited.csv")
audioset_eval = pd.read_csv("data/eval_segments-edited.csv")

In [153]:
 index_list = [
 72,
 73,
 74,
 75,
 77,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
100,
101,
102,
103,
104,
105,
106,
108,
109,
110,
111,
112,
113,
114,
115,
116,
117,
118,
119,
120,
121,
122,
123,
124,
125,
126,
127,
128,
129,
130,
131,
132,
133,
134,
135,
136 ]

In [154]:
len(index_list)

61

In [155]:
# Only use animal labels
audioset_labels = audioset_labels.iloc[index_list]
print(audioset_labels.shape)

(61, 3)


In [156]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(audioset_labels)

     index         mid                              display_name
72      72     /m/0jbk                                    Animal
73      73    /m/068hy                    Domestic animals, pets
74      74   /m/0bt9lr                                       Dog
75      75   /m/05tny_                                      Bark
77      77  /m/07qf0zm                                      Howl
79      79   /m/0ghcn6                                  Growling
80      80  /t/dd00136                             Whimper (dog)
81      81    /m/01yrx                                       Cat
82      82   /m/02yds9                                      Purr
83      83  /m/07qrkrw                                      Meow
84      84  /m/07rjwbb                                      Hiss
85      85  /m/07r81j2                                 Caterwaul
86      86    /m/0ch8v  Livestock, farm animals, working animals
87      87    /m/03k3r                                     Horse
88      88  /m/07rv9rh   

In [157]:
audioset_bal_train['singlelabel'] = audioset_bal_train[' positive_labels'].str.endswith('"')
audioset_eval['singlelabel'] = audioset_bal_train[' positive_labels'].str.endswith('"')

In [158]:
print("Train shape:", audioset_bal_train.shape)
print("Eval shape:", audioset_eval.shape)

Train shape: (22160, 16)
Eval shape: (20373, 15)


In [159]:
label_list = [ str(' "' + label) for label in audioset_labels['mid'].tolist() ]
label_list 
print(label_list)

[' "/m/0jbk', ' "/m/068hy', ' "/m/0bt9lr', ' "/m/05tny_', ' "/m/07qf0zm', ' "/m/0ghcn6', ' "/t/dd00136', ' "/m/01yrx', ' "/m/02yds9', ' "/m/07qrkrw', ' "/m/07rjwbb', ' "/m/07r81j2', ' "/m/0ch8v', ' "/m/03k3r', ' "/m/07rv9rh', ' "/m/07q5rw0', ' "/m/01xq0k1', ' "/m/07rpkh9', ' "/m/068zj', ' "/t/dd00018', ' "/m/03fwl', ' "/m/07q0h5t', ' "/m/07bgp', ' "/m/025rv6n', ' "/m/09b5t', ' "/m/07st89h', ' "/m/07qn5dc', ' "/m/01rd7k', ' "/m/07svc2k', ' "/m/09ddx', ' "/m/07qdb04', ' "/m/0dbvp', ' "/m/01280g', ' "/m/0cdnk', ' "/m/04cvmfc', ' "/m/015p6', ' "/m/020bb7', ' "/m/07pggtn', ' "/m/07sx8x_', ' "/m/0h0rv', ' "/m/07r_25d', ' "/m/04s8yn', ' "/m/07r5c2p', ' "/m/09d5_', ' "/m/07r_80w', ' "/m/05_wcq', ' "/m/01z5f', ' "/m/06hps', ' "/m/04rmv', ' "/m/07r4gkf', ' "/m/03vt0', ' "/m/09xqv', ' "/m/09f96', ' "/m/0h2mp', ' "/m/07pjwq1', ' "/m/01h3n', ' "/m/09ld4', ' "/m/07st88b', ' "/m/078jl', ' "/m/07qn4z3', ' "/m/032n05']


In [160]:
# Fitler all non-animal data
audioset_bal_train = audioset_bal_train[audioset_bal_train[' positive_labels'].isin(label_list)]
print("Balanced train data:")
print(audioset_bal_train.shape)
print(audioset_bal_train.head())

#  print("Unbalanced train data:")
#  audioset_unbal_train[audioset_unbal_train[' positive_labels'].isin(label_list)]
#  audioset_unbal_train.shape

audioset_eval = audioset_eval[audioset_eval[' positive_labels'].isin(label_list)]
print("Eval data")
print(audioset_eval.shape)
print(audioset_eval.head())

Balanced train data:
(1664, 16)
         # YTID   start_seconds   end_seconds  positive_labels  Unnamed: 4  \
2        #NAME?             0.0          10.0        "/m/03fwl    /m/04rlf   
29  -3IYpJfLVJk            30.0          40.0        "/m/03k3r  /m/07q5rw0   
54  -6x2PtSRfJU            30.0          40.0        "/m/01yrx    /m/068hy   
64  -7ye24UFRng             0.0          10.0       "/m/032n05   /m/04rlf"   
86       #NAME?            30.0          40.0        "/m/07bgp  /m/07q0h5t   

    Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  \
2    /m/09x0r"        NaN        NaN        NaN        NaN         NaN   
29    /m/0jbk"        NaN        NaN        NaN        NaN         NaN   
54  /m/07qrkrw  /m/09x0r"        NaN        NaN        NaN         NaN   
64         NaN        NaN        NaN        NaN        NaN         NaN   
86   /m/09x0r"        NaN        NaN        NaN        NaN         NaN   

   Unnamed: 11 Unnamed: 12 Unnamed: 13 Unnamed: 14  si

In [161]:
# Fitler all wavs with multiple labels
#audioset_bal_train = audioset_bal_train.loc[audioset_bal_train['multilabel'] == False]
#print("Balanced train data:")
#print(audioset_bal_train.shape)
#print(audioset_bal_train.head())

#  print("Unbalanced train data:")
#  audioset_unbal_train[audioset_unbal_train[' positive_labels'].isin(label_list)]
#  audioset_unbal_train.shape

#audioset_eval = audioset_eval.loc[audioset_eval['multilabel'] == False]
#print("Eval data")
#print(audioset_eval.shape)
#print(audioset_eval.head())

In [162]:
# Save filtered data frames
audioset_bal_train.to_csv('data/balanced_train_segments-animals.csv', header=True, index=False)
audioset_eval.to_csv('data/eval_segments-animals.csv', header=True, index=False)
audioset_labels.to_csv('data/class_labels_indices-animals.csv', header=True, index=False)

In [163]:
audioset_bal_train.columns

Index(['# YTID', ' start_seconds', ' end_seconds', ' positive_labels',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'singlelabel'],
      dtype='object')

In [164]:
def supported(name):
    url = "https://youtube.com/watch?v=" + name
    try:
        youtube = pytube.YouTube(url)
    except:
        return False
    else:
        return True

In [165]:
# Probably don't need to do this! It takes a very long time. Better of just listing whatever wav files you do have
# after running the download script and filtering the dataframe then

In [166]:
# Check all videso to see if they no longer exist on youtube.com
#audioset_bal_train['exists'] = audioset_bal_train['# YTID'].apply(supported)

# Check all videso to see if they no longer exist on youtube.com
#audioset_eval['exists'] = audioset_eval['# YTID'].apply(supported)

In [167]:
#audioset_bal_train.head(30)

In [168]:
# Filter videos no longer on YouTube
#audioset_bal_train = audioset_bal_train.loc[audioset_bal_train['exists'] == True]
#print("Balanced train data:")
#print(audioset_bal_train.shape)

#  audioset_ubbal_train = audioset_ubbal_train.loc[audioset_ubbal_train['exists'] == True]
#  print("Unbalanced train data:")
#  print(audioset_unbal_train.shape)

#audioset_eval = audioset_eval.loc[audioset_eval['exists'] == True]
#print("Eval data")
#print(audioset_eval.shape)

In [169]:
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [170]:
audioset_labels.set_index('mid')

,index,display_name
mid,,
/m/0jbk,72,Animal
/m/068hy,73,"Domestic animals, pets"
/m/0bt9lr,74,Dog
/m/05tny_,75,Bark
/m/07qf0zm,77,Howl
/m/0ghcn6,79,Growling
/t/dd00136,80,Whimper (dog)
/m/01yrx,81,Cat
/m/02yds9,82,Purr


In [171]:
def get_display_name_from_mid(mid):
    return audioset_labels[mid]

In [172]:
# counts = audioset_bal_train[' positive_labels'].value_counts()
# label_counts = pd.DataFrame({'label':audioset_bal_train[' positive_labels'], 'count':audioset_bal_train[' positive_labels'].value_counts()})
# prices.hist()

df = audioset_bal_train[' positive_labels'].value_counts().to_frame()
df.reset_index(inplace=True)
df.columns = [' mid', ' positive_labels']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

# df['display_name'] = df[' mid'].apply(get_display_name_from_mid)

             mid   positive_labels
0      "/m/015p6               171
1      "/m/068hy               157
2    "/m/025rv6n               133
3      "/m/01yrx               120
4      "/m/03vt0               104
5      "/m/03k3r                77
6    "/m/01xq0k1                62
7     "/m/01280g                60
8      "/m/01h3n                60
9    "/m/07st88b                59
10   "/m/07r_80w                58
11     "/m/01z5f                57
12     "/m/03fwl                56
13    "/m/01rd7k                47
14    "/m/04s8yn                42
15     "/m/068zj                42
16     "/m/078jl                38
17     "/m/07bgp                37
18    "/m/05tny_                36
19    "/m/032n05                29
20   "/m/04cvmfc                28
21   "/m/07qdb04                27
22     "/m/04rmv                26
23   "/m/07pjwq1                19
24     "/m/06hps                17
25   "/m/07sx8x_                16
26    "/m/02yds9                15
27   "/m/07rv9rh    